In [1]:
import numpy

from finance_utils.namedtuples import *
from finance_utils.uniontypes import *
from finance_utils.asset.spot.base import Spot,ServiceChargesSpot
# from finance_utils.asset.spot.strategy import SpotCostAveragingPlan
from finance_utils.strategy.dca import simple_dca, price_qualified_dca

from finance_utils.process.prices.gbm import RVDecoupledGBM, GBM, NormalDistribution
from finance_utils.trader.base import Trader
from finance_utils.trader.utility import *

from matplotlib import pyplot

from data_utils.stochastic_utils.distributions.nonParametricDistribution import HistogramDist, KernelMixDist, SmoothHisDist
from data_utils.stochastic_utils.random_process.correlatedRandom import random_correlated_series
from easy_datetime.timestamp import TimeStamp
import tqdm

from matplotlib import pyplot

In [ ]:
class Test:
    def __init__(
            self, 
            # cap=20000 * 12, 
            # rr=(0.03, 0.03), 
            # v=(0.015,0.015), 
            # corr=(0,0), 
            weekdays=22,
            months=12,
    ):
        # self.cap = cap
        # self.rr = rr
        # self.v = v
        # self.corr = corr
        self.weekdays = weekdays
        self.months = months
        self.days = self.weekdays * self.months
        self.stocks:list[Spot] = []
        self.processes:list[RVDecoupledGBM] = []
        
    def new_processes(self, rr=(0.03, 0.03), v=(0.015, 0.015), corr=(0,0)):
        d = []
        for i in range(len(rr)):
            d.append(NormalDistribution(rr[i], v[i]))
        rs = random_correlated_series(d, corr, self.days)
        for i in range(len(rr)):
            self.stocks.append(
                ServiceChargesSpot(f"{i}", 1, TradeDelta("day", 1), 5e-5, 5e-4)
            )
            self.processes.append(
                RVDecoupledGBM(f"{i}", rs[i], 1, TimeStamp.now().accurate_to("year"), "day", 1)
            )
        
        
            
    def trade(self, trade_interval = 1):
        index_list = []
        st_list = []
        pt_list = []
        trigger_set = range(0, self.days, trade_interval)
        
        
        num = len(self.stocks)
        simple_traders = [Trader(f"s_trader_{i}", 200000, None) for i in range(num)]
        price_qualified_traders = [Trader(f"p_trader_{i}", 200000, None) for i in range(num)]
        
        for i, t, _ in self.processes[0].process_iter():
            dic = {}
            index = 0
            for n in range(num):
                path = self.processes[n][i]
                # root = list(path.dic)[0]
                # dic[root] = path.dic[root]
                index += path.price
                _, _ = simple_dca(self.processes[n].s0, i, t, path, self.stocks[n], simple_traders[n], cap, trigger_set)
                _, _ = price_qualified_dca(self.processes[n].s0, i, t, path, self.stocks[n], price_qualified_traders[n], cap2, trigger_set, pr, sr)
                
            
                
            
